In [1]:
import pandas as pd
from g4f.client import Client
import re

In [2]:
def load_data(file_path: str) -> pd.DataFrame:
    """
    加載 CSV 文件並處理錯誤行。
    """
    try:
        return pd.read_csv(file_path, on_bad_lines="skip", encoding="utf-8")
    except pd.errors.ParserError as e:
        raise Exception(f"CSV 文件解析錯誤：{e}")
    except Exception as e:
        raise Exception(f"讀取數據時發生錯誤：{e}")



In [3]:
def generate_summary(data: pd.DataFrame):
    """
    生成數據描述性統計。
    Args:
        data (pd.DataFrame): 輸入數據框。
    Returns:
        pd.DataFrame: 描述性統計。
    """
    try:
        return data.describe()
    except Exception as e:
        raise Exception(f"生成描述性統計時發生錯誤：{e}")

In [4]:
def construct_prompt(user_input: str, summary_stats: pd.DataFrame) -> tuple:
    """
    構建系統與用戶提示。
    Args:
        user_input (str): 用戶查詢。
        summary_stats (pd.DataFrame): 描述性統計。
    Returns:
        tuple: (系統提示, 用戶提示)
    """
    system_prompt = (
        "你是一位專業的數據科學家，擅長分析數據並從中提取見解。"
        "以下是一些統計數據，請根據這些數據提供深入分析和建議：\n"
        "1. 描述數據的整體特性，例如平均值、標準差、數據分佈等。\n"
        "2. 尋找可能的異常值或數據問題，並評估其對分析結果的影響。\n"
        "3. 如果有類別型數據，請分析其分佈特性和可能的相關性。\n"
        "4. 根據數據，提供有助於業務決策的具體建議，例如潛在的改進方向或應關注的特定方向。\n"
        "5. 將您的分析結論簡潔清楚地表達，並建議下一步的數據收集或分析方向。"
    )
    user_prompt = (
        f"以下是數據的描述性統計結果：\n{summary_stats.to_string()}\n"
        f"請根據這些數據回答以下問題：{user_input}"
    )
    return system_prompt, user_prompt


In [5]:
def generate_report(file_path: str, user_input: str) -> str:
    """
    基於自然語言查詢生成分析報告。
    Args:
        file_path (str): 數據文件路徑。
        user_input (str): 用戶的自然語言查詢。
    Returns:
        str: 分析報告或錯誤信息。
    """
    try:
        # 1. 加載數據
        data = load_data(file_path)

        # 2. 生成描述性統計
        summary_stats = generate_summary(data)

        # 3. 構建 GPT 模型請求的提示
        system_prompt, user_prompt = construct_prompt(user_input, summary_stats)

        # 4. 初始化 GPT 客戶端並生成報告
        client = Client()
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.1,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"生成報告時發生錯誤：{e}"



In [7]:
if __name__ == "__main__":
    # 示例使用
    file_path = file_path = r".\csv.ingredient\tsmc_operating_profit_margin_2010_2023.csv"

    user_query = "請分析數據並提供詳細的見解和建議，並提供報告。"  # 自然語言查詢

    try:
        report = generate_report(file_path, user_query)
        print("生成的報告：")
        print(report)
    except Exception as e:
        print(f"程序運行失敗：{e}")

生成的報告：
### 數據分析報告

#### 1. 整體特性描述
根據提供的描述性統計結果，我們可以對數據進行以下分析：

- **年份分佈**：
  - 數據範圍從2010年到2023年，共56個觀測值，顯示出數據的時間跨度涵蓋了13年。
  - 平均年份為2016.5，這表明數據的中心趨向於中間年份。

- **經營利潤率**：
  - 平均經營利潤率為38.38%，顯示出整體的盈利能力相對穩定。
  - 標準差為2.33，表明經營利潤率的變異性較小，數據相對集中。
  - 最小值為33.6%，最大值為42.81%，顯示出經營利潤率在不同年份之間存在一定的波動。

- **四分位數**：
  - 第一四分位數（25%）為36.39%，中位數（50%）為38.63%，第三四分位數（75%）為40.09%。這表明大多數數據集中在36.39%到40.09%之間。

#### 2. 異常值和數據問題
在這組數據中，經營利潤率的範圍相對狹窄，且標準差不大，這表明數據中可能不存在明顯的異常值。然而，為了進一步確認，可以使用箱型圖來視覺化數據分佈，檢查是否存在潛在的異常值。

#### 3. 類別型數據分析
在這組數據中，年份作為類別型數據，雖然它是連續的，但可以將其視為類別進行分析。可以進一步分析每一年經營利潤率的變化趨勢，並檢查是否存在某些年份的利潤率顯著高於或低於其他年份。

#### 4. 業務決策建議
根據數據分析，以下是幾個具體的建議：

- **持續監控經營利潤率**：由於經營利潤率在過去幾年中顯示出穩定的增長趨勢，建議企業持續監控這一指標，並分析影響其變化的內外部因素。
  
- **深入分析高利潤年份**：特別關注2020年和2023年的經營利潤率，這些年份的利潤率相對較高，應分析其背後的原因，並考慮是否可以將這些成功的策略應用到其他年份。

- **風險管理**：儘管目前的利潤率表現良好，但仍需考慮市場變化和潛在的經濟風險，建議制定相應的風險管理策略。

#### 5. 下一步建議
- **數據收集**：建議收集更多的數據，包括影響經營利潤率的其他指標（如銷售額、成本結構等），以便進行更全面的分析。
  
- **進一步分析**：可以進行時間序列分析，檢查經營利潤率的趨勢和季節性變化，並預測未來的利潤率走勢。

- **視覺化數據**：使用圖表（如折線圖